In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## การสร้าง Client

ในตัวอย่างนี้ เราจะใช้ [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) เพื่อเข้าถึง LLM

`model` ถูกกำหนดเป็น `gpt-4o-mini` ลองเปลี่ยน model ไปเป็น model อื่นที่มีอยู่ในตลาด GitHub Models เพื่อดูผลลัพธ์ที่แตกต่างกัน

เพื่อการทดสอบอย่างรวดเร็ว เราจะลองรัน prompt ง่ายๆ - `What is the capital of France`


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## การกำหนดฟังก์ชัน

ในตัวอย่างนี้ เราจะให้ตัวแทนเข้าถึงเครื่องมือที่เป็นฟังก์ชันซึ่งมีรายการจุดหมายปลายทางสำหรับการพักผ่อนและข้อมูลความพร้อมใช้งานของแต่ละที่

คุณสามารถนึกภาพว่านี่เป็นสถานการณ์ที่ตัวแทนท่องเที่ยวอาจมีการเข้าถึงฐานข้อมูลการเดินทางเป็นต้น

ขณะที่คุณศึกษาตัวอย่างนี้ ลองกำหนดฟังก์ชันและเครื่องมือใหม่ๆ ที่ตัวแทนสามารถเรียกใช้งานได้ตามต้องการ


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## การกำหนด Function Tool  
เพื่อให้ตัวแทนสามารถใช้ `vacation_destinations` เป็น `FunctionTool` ได้ เราจำเป็นต้องกำหนดให้มันเป็นหนึ่งในนั้น  

เรายังต้องให้คำอธิบายเกี่ยวกับเครื่องมือนี้ด้วย ซึ่งจะช่วยให้ตัวแทนเข้าใจว่าเครื่องมือนี้ใช้ทำอะไรในบริบทของงานที่ผู้ใช้ร้องขอ


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## การกำหนดตัวแทน

ตอนนี้เราสามารถสร้างตัวแทนได้ในโค้ดด้านล่าง เรากำหนด `system_message` เพื่อให้คำแนะนำแก่ตัวแทนเกี่ยวกับวิธีช่วยผู้ใช้ค้นหาสถานที่ท่องเที่ยวสำหรับวันหยุดพักผ่อน

เรายังตั้งค่าพารามิเตอร์ `reflect_on_tool_use` เป็น true ซึ่งช่วยให้ LLM ใช้ผลลัพธ์จากการเรียกใช้เครื่องมือและส่งคำตอบในรูปแบบภาษาธรรมชาติ

คุณสามารถตั้งค่าพารามิเตอร์นี้เป็น false เพื่อดูความแตกต่าง


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## การใช้งาน Agent

ตอนนี้เราสามารถใช้งาน Agent ได้แล้ว โดยเริ่มจากข้อความแรกของผู้ใช้ที่ถามเกี่ยวกับการเดินทางไปโตเกียว

คุณสามารถเปลี่ยนจุดหมายปลายทางของเมืองนี้เพื่อดูว่า Agent ตอบสนองต่อความพร้อมของเมืองนั้นอย่างไร


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้องมากที่สุด แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามืออาชีพ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
